In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE5667'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcription data from Normal Skin and Nonlesional and Lesional Atopic Dermatitis/Eczema Skin"
!Series_summary	"Atopic dermatitis (AD) is a common pruritic dermatitis with macroscopically nonlesional skin that is often abnormal.  Therefore, we used high-density oligonucleotide arrays to identify cutaneous gene transcription changes associated with early AD inflammation as potential disease control targets.  Skin biopsy specimens analyzed included normal skin from five healthy nonatopic adults and both minimally lesional skin and nearby or contralateral nonlesional skin from six adult AD patients."
!Series_summary	"Keywords: disease state analysis"
!Series_overall_design	"We used high-density oligonucleotide Affymetrix Human U133A GeneChip arrays to identify cutaneous gene transcription changes associated with early AD inflammation as potential disease control targets.  Skin biopsy specimens analyzed included normal skin from five healthy nonatopi

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Checking gene expression availability
series_summary = "!Series_summary\t\"Atopic dermatitis (AD) is a common pruritic dermatitis with macroscopically nonlesional skin that is often abnormal.  Therefore, we used high-density oligonucleotide arrays to identify cutaneous gene transcription changes associated with early AD inflammation as potential disease control targets.  Skin biopsy specimens analyzed included normal skin from five healthy nonatopic adults and both minimally lesional skin and nearby or contralateral nonlesional skin from six adult AD patients.\""
series_overall_design = "!Series_overall_design\t\"We used high-density oligonucleotide Affymetrix Human U133A GeneChip arrays to identify cutaneous gene transcription changes associated with early AD inflammation as potential disease control targets.  Skin biopsy specimens analyzed included normal skin from five healthy nonatopic adults and both minimally lesional skin and nearby or contralateral nonlesional skin from six adult AD patients.\""

if "Affymetrix Human U133A GeneChip arrays" in series_summary or series_overall_design:
    is_gene_available = True

# Extracting variable information from the sample characteristics
sample_characteristics_dict = {
    0: ['Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:normal_Age:35years_Gender:female_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:normal_Age:26years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:normal_Age:28years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:normal_Age:22years_Gender:female_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper leg_Health status:normal_Age:32years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:atopic dermatitis_Disease Severity:severe_Skin status:nonlesional_Age:44years_Gender:female_Race:black', 
        'Tissue:skin(dermis and epidermis)_Source:upper leg_Health status:atopic dermatitis_Disease Severity:severe_Skin status:nonlesional_Age:19years_Gender:male_Race:hispanic', 
        'Tissue:skin(dermis and epidermis)_Source:mid arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:nonlesional_Age:18years_Gender:male_Race:asian', 
        'Tissue:skin(dermis and epidermis)_Source:upper back_Health status:atopic dermatitis_Disease Severity:severe_Skin status:nonlesional_Age:38years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:mid arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:nonlesional_Age:32years_Gender:female_Race:asian', 
        'Tissue:skin(dermis and epidermis)_Source:lower arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:nonlesional_Age:39years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:upper arm_Health status:atopic dermatitis_Disease Severity:severe_Skin status:lesional_Age:44years_Gender:female_Race:black', 
        'Tissue:skin(dermis and epidermis)_Source:upper leg_Health status:atopic dermatitis_Disease Severity:severe_Skin status:lesional_Age:19years_Gender:male_Race:hispanic', 
        'Tissue:skin(dermis and epidermis)_Source:mid arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:lesional_Age:18years_Gender:male_Race:asian', 
        'Tissue:skin(dermis and epidermis)_Source:upper back_Health status:atopic dermatitis_Disease Severity:severe_Skin status:lesional_Age:38years_Gender:male_Race:white', 
        'Tissue:skin(dermis and epidermis)_Source:mid arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:lesional_Age:32years_Gender:female_Race:asian', 
        'Tissue:skin(dermis and epidermis)_Source:lower arm_Health status:atopic dermatitis_Disease Severity:mild/moderate_Skin status:lesional_Age:39years_Gender:male_Race:white']
}

# Searching for trait, age, and gender information
for key, value in sample_characteristics_dict.items():
    if 'Health status:' in value[0]:
        trait_row = key
    if 'Age:' in value[0]:
        age_row = key
    if 'Gender:' in value[0]:
        gender_row = key

# Define conversion functions
def convert_trait(value):
    try:
        trait_value = value.split('Health status:')[1].split('_')[0].strip()
        return 1 if trait_value.lower() == 'atopic dermatitis' else 0
    except:
        return None

def convert_age(value):
    try:
        age_value = value.split('Age:')[1].split('years')[0].strip()
        return float(age_value)
    except:
        return None

def convert_gender(value):
    try:
        gender_value = value.split('Gender:')[1].split('_')[0].strip()
        return 1 if gender_value.lower() == 'male' else 0
    except:
        return None

# Save cohort information
save_cohort_info('GSE5667', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Eczema/trait_data/GSE5667.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM132623': [0.0, 35.0, 0.0], 'GSM132624': [0.0, 26.0, 1.0], 'GSM132625': [0.0, 28.0, 1.0], 'GSM132626': [0.0, 22.0, 0.0], 'GSM132627': [0.0, 32.0, 1.0], 'GSM132628': [1.0, 44.0, 0.0], 'GSM132629': [1.0, 19.0, 1.0], 'GSM132630': [1.0, 18.0, 1.0], 'GSM132631': [1.0, 38.0, 1.0], 'GSM132632': [1.0, 32.0, 0.0], 'GSM132633': [1.0, 39.0, 1.0], 'GSM132634': [1.0, 44.0, 0.0], 'GSM132635': [1.0, 19.0, 1.0], 'GSM132636': [1.0, 18.0, 1.0], 'GSM132637': [1.0, 38.0, 1.0], 'GSM132638': [1.0, 32.0, 0.0], 'GSM132639': [1.0, 39.0, 1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_data = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_data)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Eczema/gene_data/GSE5667.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Eczema')

# 4. Save the cohort information.
save_cohort_info('GSE5667', './preprocessed/Eczema/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Eczema/GSE5667.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Eczema', the least common label is '0.0' with 5 occurrences. This represents 29.41% of the dataset.
The distribution of the feature 'Eczema' in this dataset is fine.

Quartiles for 'Age':
  25%: 22.0
  50% (Median): 32.0
  75%: 38.0
Min: 18.0
Max: 44.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 6 occurrences. This represents 35.29% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

